In [0]:
#===从GCS加载数据
import warnings
warnings.filterwarnings("ignore")

project_id = 'amazing-firefly-153908'
bucket_name = 'nlp_final'

# 获取授权
from google.colab import auth
auth.authenticate_user()

#从GCS下载数据至tmp
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')
from apiclient.http import MediaIoBaseDownload

# with open('/tmp/train.csv', 'wb') as f:
#   request = gcs_service.objects().get_media(bucket=bucket_name,
#                                             object='train.csv')
#   media = MediaIoBaseDownload(f, request)

#   done = False
#   while not done:
#     # _ is a placeholder for a progress object that we ignore.
#     # (Our file is small, so we skip reporting progress.)
#     _, done = media.next_chunk()

# with open('/tmp/test.csv', 'wb') as f:
#   request = gcs_service.objects().get_media(bucket=bucket_name,
#                                             object='test.csv')
#   media = MediaIoBaseDownload(f, request)

#   done = False
#   while not done:
#     # _ is a placeholder for a progress object that we ignore.
#     # (Our file is small, so we skip reporting progress.)
#     _, done = media.next_chunk()
    
# print('Download complete')

W0819 06:50:02.424576 140022735243136 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0819 06:50:03.619950 140022735243136 __init__.py:44] file_cache is unavailable when using oauth2client >= 4.0.0 or google-auth
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ModuleNotFoundError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/googleapiclient/discovery_cache/file_cache.py

In [0]:
import numpy as np
import pandas as pd
import re
from string import punctuation

# train = pd.read_csv('/tmp/train.csv')
# test = pd.read_csv('/tmp/test.csv')
# train.head()

In [0]:
# x_train = train[['id','question1', 'question2']]
# x_test = test
# x_train.head()

,id,question1,question2
0,0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [0]:
#转为小写、清除特殊符号、将一些常见同义词转换统一、去除标点
#来自 https://www.kaggle.com/benjaminkz/quora-question-pairs-xgboost/notebook

def common_words_transformation_remove_punctuation(text):
    
    text = text.lower()
    
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"who's", "who is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"when's", "when is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"there's", "there is", text)

    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"\'s", " ", text)  # 除了上面的特殊情况外，“\'s”只能表示所有格，应替换成“ ”
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r" m ", " am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"60k", " 60000 ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e-mail", "email", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"quikly", "quickly", text)
    text = re.sub(r" usa ", " america ", text)
    text = re.sub(r" u s ", " america ", text)
    text = re.sub(r" uk ", " england ", text)
    text = re.sub(r"imrovement", "improvement", text)
    text = re.sub(r"intially", "initially", text)
    text = re.sub(r" dms ", "direct messages ", text)  
    text = re.sub(r"demonitization", "demonetization", text) 
    text = re.sub(r"actived", "active", text)
    text = re.sub(r"kms", " kilometers ", text)
    text = re.sub(r" cs ", " computer science ", text)
    text = re.sub(r" ds ", " data science ", text)
    text = re.sub(r" ee ", " electronic engineering ", text)
    text = re.sub(r" upvotes ", " up votes ", text)
    text = re.sub(r" iphone ", " phone ", text)
    text = re.sub(r"\0rs ", " rs ", text) 
    text = re.sub(r"calender", "calendar", text)
    text = re.sub(r"ios", "operating system", text)
    text = re.sub(r"programing", "programming", text)
    text = re.sub(r"bestfriend", "best friend", text)
    text = re.sub(r"III", "3", text) 
    text = re.sub(r"the us", "america", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r"\0s", "0", text)
    
    text = "".join([c for c in text if c not in punctuation])
        
    return text

In [0]:
# x_train = x_train.fillna(" ")
# x_test = x_test.fillna(" ")

# x_train['question1'] = x_train['question1'].apply(common_words_transformation_remove_punctuation)
# x_train['question2'] = x_train['question2'].apply(common_words_transformation_remove_punctuation)
# x_test['question1'] = x_test['question1'].apply(common_words_transformation_remove_punctuation)
# x_test['question2'] = x_test['question2'].apply(common_words_transformation_remove_punctuation)

# x_train.to_csv("/tmp/x_train.csv", index = False)
# x_test.to_csv("/tmp/x_test.csv", index = False)

# x_train.head()

,id,question1,question2
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,1,what is the story of kohinoor koh i noor diamond,what would happen if the indian government sto...
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,3,why am i mentally very lonely how can i solve it,find the remainder when math 23 24 math is d...
4,4,which one dissolve in water quickly sugar sal...,which fish would survive in salt water


In [0]:
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from tqdm.auto import tqdm
tqdm.pandas()

import time

def get_time():
  seconds = time.time()
  named_tuple = time.gmtime(seconds) # get struct_time
  hour = named_tuple.tm_hour + 8
  time_str = str(hour) + ':' + str(named_tuple.tm_min) + ':' + str(named_tuple.tm_sec)
  return time_str

#stemming
def stm(tokens):
  stemmer = nltk.stem.PorterStemmer()
  return [stemmer.stem(token) for token in tokens]

#lemmatize
def get_wordnet_pos(treebank_tag):
  """
  return WORDNET POS compliance to WORDENT lemmatization (a,n,r,v) 
  """
  if treebank_tag.startswith('J'):
    return wordnet.ADJ
  elif treebank_tag.startswith('V'):
    return wordnet.VERB
  elif treebank_tag.startswith('N'):
    return wordnet.NOUN
  elif treebank_tag.startswith('R'):
    return wordnet.ADV
  else:
    # As default pos in lemmatization is Noun
    return wordnet.NOUN


def lmtz(tokens):
  lemmatizer = nltk.stem.WordNetLemmatizer()
  # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
  pos_tokens = nltk.pos_tag(tokens)
  return [lemmatizer.lemmatize(word,get_wordnet_pos(pos_tag)) for (word, pos_tag) in pos_tokens]

#stop_words
from nltk.corpus import stopwords
def stwd(tokens):
  stop_words = set(stopwords.words("english"))
  return [token for token in tokens if not token in stop_words]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [0]:

with open('/tmp/x_test.csv', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='test.csv')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()
    
print('Download complete')

x_test = pd.read_csv('/tmp/x_test.csv')

x_test.head()

Download complete


,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?


In [0]:
# x_train_tk = x_train

# del x_train

# x_train_tk['question1'] = x_train_tk['question1'].apply(nltk.word_tokenize)
# x_train_tk['question2'] = x_train_tk['question2'].apply(nltk.word_tokenize)

# x_train_tk.to_csv("/tmp/x_train_tk.csv", index = False)
# print("x_train tokenized")

# x_train_stm = x_train_tk
# x_train_stm['question1'] = x_train_stm['question1'].apply(stm)
# x_train_stm['question2'] = x_train_stm['question2'].apply(stm)

# x_train_stm.to_csv("/tmp/x_train_stm.csv", index = False)
# print("x_train stemming complete")

# x_train_lmtz = x_train_tk
# x_train_lmtz['question1'] = x_train_lmtz['question1'].apply(stm)
# x_train_lmtz['question2'] = x_train_lmtz['question2'].apply(stm)

# x_train_lmtz.to_csv("/tmp/x_train_lmtz.csv", index = False)
# print("x_train lemmatizing complete")

# x_train_stwd = x_train_tk
# x_train_stwd['question1'] = x_train_stwd['question1'].apply(stm)
# x_train_stwd['question2'] = x_train_stwd['question2'].apply(stm)

# x_train_stwd.to_csv("/tmp/x_train_stwd.csv", index = False)
# print("x_train remove stop words complete")

# del x_train_tk, x_train_stm, x_train_lmtz

# x_train_stwd_stm = x_train_stwd
# x_train_stwd_stm['question1'] = x_train_stwd_stm['question1'].apply(stm)
# x_train_stwd_stm['question2'] = x_train_stwd_stm['question2'].apply(stm)

# x_train_stwd_stm.to_csv("/tmp/x_train_stwd_stm.csv", index = False)
# print("x_train stop words and stemming complete")

# x_train_stwd_lmtz = x_train_stwd
# x_train_stwd_lmtz['question1'] = x_train_stwd_lmtz['question1'].apply(lmtz)
# x_train_stwd_lmtz['question2'] = x_train_stwd_lmtz['question2'].apply(lmtz)

# x_train_stwd_lmtz.to_csv("/tmp/x_train_stwd_lmtz.csv", index = False)
# print("x_train stop words and lemmatizing complete")

# del x_train_stwd, x_train_stwd_stm, x_train_stwd_lmtz

# print("x_train pre-processing complete")

x_train tokenized
x_train stemming complete
x_train lemmatizing complete
x_train remove stop words complete
x_train stop words and stemming complete
x_train stop words and lemmatizing complete
x_train pre-processing complete


In [0]:
x_test = x_test.fillna(" ")

time_str = get_time()
print("x_test cleaning start at " + time_str)

x_test['question1'] = x_test['question1'].progress_apply(common_words_transformation_remove_punctuation)
x_test['question2'] = x_test['question2'].progress_apply(common_words_transformation_remove_punctuation)
print("x_test cleaned")
x_test.head()

x_test cleaning start at 15:9:29



x_test cleaned


,test_id,question1,question2
0,0,how does the surface pro himself 4 compare wit...,why did microsoft choose core m3 and not core ...
1,1,should i have a hair transplant at age 24 how ...,how much cost does hair transplant require
2,2,what but is the best way to send money from ch...,what you send money to china
3,3,which food not emulsifiers,what foods fibre
4,4,how aberystwyth start reading,how their can i start reading


In [0]:
x_test_stm = x_test

def stm(text):
  tokens = nltk.word_tokenize(text)
  stemmer = nltk.stem.PorterStemmer()
  return " ".join([stemmer.stem(token) for token in tokens])

time_str = get_time()
print("x_test stemming start at " + time_str)
x_test_stm['question1'] = x_test_stm['question1'].progress_apply(stm)
x_test_stm['question2'] = x_test_stm['question2'].progress_apply(stm)
x_test_stm.head()

x_test stemming start at 15:17:29


,test_id,question1,question2
0,0,how doe the surfac pro himself 4 compar with i...,whi did microsoft choos core m3 and not core i...
1,1,should i have a hair transplant at age 24 how ...,how much cost doe hair transplant requir
2,2,what but is the best way to send money from ch...,what you send money to china
3,3,which food not emulsifi,what food fibr
4,4,how aberystwyth start read,how their can i start read


In [0]:
x_test_stm.to_csv("/tmp/x_test_stm_cvt.csv", index = False)
upload_to_GCS('x_test_stm_cvt.csv')

x_test_stm_cvt.csv uploaded


In [0]:
x_test_lmtz = x_test

def lmtz(text):
  tokens = nltk.word_tokenize(text)
  lemmatizer = nltk.stem.WordNetLemmatizer()
  # find the pos tagginf for each tokens [('What', 'WP'), ('can', 'MD'), ('I', 'PRP') ....
  pos_tokens = nltk.pos_tag(tokens)
  return [lemmatizer.lemmatize(word,get_wordnet_pos(pos_tag)) for (word, pos_tag) in pos_tokens]

time_str = get_time()
print("x_test lemmatizmming start at " + time_str)
x_test_lmtz['question1'] = x_test_lmtz['question1'].progress_apply(lmtz)
x_test_lmtz['question2'] = x_test_lmtz['question2'].progress_apply(lmtz)
x_test_lmtz.head()

x_test lemmatizmming start at 16:1:59


,test_id,question1,question2
0,0,"[how, doe, the, surfac, pro, himself, 4, compa...","[whi, do, microsoft, choos, core, m3, and, not..."
1,1,"[should, i, have, a, hair, transplant, at, age...","[how, much, cost, doe, hair, transplant, requir]"
2,2,"[what, but, be, the, best, way, to, send, mone...","[what, you, send, money, to, china]"
3,3,"[which, food, not, emulsifi]","[what, food, fibr]"
4,4,"[how, aberystwyth, start, read]","[how, their, can, i, start, read]"


In [0]:
x_test_lmtz.to_csv("/tmp/x_test_lmtz_cvt.csv", index = False)
upload_to_GCS('x_test_lmtz_cvt.csv')

x_test_lmtz_cvt.csv uploaded


In [0]:
x_test = x_test.fillna(" ")

time_str = get_time()
print("x_test cleaning start at " + time_str)

x_test['question1'] = x_test['question1'].progress_apply(common_words_transformation_remove_punctuation)
x_test['question2'] = x_test['question2'].progress_apply(common_words_transformation_remove_punctuation)
print("x_test cleaned")

x_test_tk = x_test

del x_test

time_str = get_time()
print("x_test tokenizing start at " + time_str)

x_test_tk['question1'] = x_test_tk['question1'].progress_apply(nltk.word_tokenize)
x_test_tk['question2'] = x_test_tk['question2'].progress_apply(nltk.word_tokenize)

x_test_tk.to_csv("/tmp/x_test_tk.csv", index = False)
print("x_test tokenized")

x_test_stm = x_test_tk
x_test_stm['question1'] = x_test_stm['question1'].apply(stm)
x_test_stm['question2'] = x_test_stm['question2'].apply(stm)

x_test_stm.to_csv("/tmp/x_test_stm.csv", index = False)
del x_test_stm
print("x_test stemming complete")

x_test_lmtz = x_test_tk
x_test_lmtz['question1'] = x_test_lmtz['question1'].apply(stm)
x_test_lmtz['question2'] = x_test_lmtz['question2'].apply(stm)

x_test_lmtz.to_csv("/tmp/x_test_lmtz.csv", index = False)
del x_test_lmtz
print("x_test lemmatizing complete")

x_test_stwd = x_test_tk

time_str = get_time()
print("x_test stop words removing start at " + time_str)

x_test_stwd['question1'] = x_test_stwd['question1'].progress_apply(stm)
x_test_stwd['question2'] = x_test_stwd['question2'].progress_apply(stm)

x_test_stwd.to_csv("/tmp/x_test_stwd.csv", index = False)
print("x_test remove stop words complete")

del x_test_tk

# x_test_stwd_stm = x_test_stwd
# x_test_stwd_stm['question1'] = x_test_stwd_stm['question1'].apply(stm)
# x_test_stwd_stm['question2'] = x_test_stwd_stm['question2'].apply(stm)

# x_test_stwd_stm.to_csv("/tmp/x_test_stwd_stm.csv", index = False)
# del x_test_stwd_stm
# print("x_test stop words and stemming complete")

x_test_stwd_lmtz = x_test_stwd
time_str = get_time()
print("x_test stop words and lemmatizing start at " + time_str)
x_test_stwd_lmtz['question1'] = x_test_stwd_lmtz['question1'].progress_apply(lmtz)
x_test_stwd_lmtz['question2'] = x_test_stwd_lmtz['question2'].progress_apply(lmtz)

x_test_stwd_lmtz.to_csv("/tmp/x_test_stwd_lmtz.csv", index = False)
del x_test_stwd_lmtz
time_str = get_time()
print("x_test stop words and lemmatizing complete at "+ time_str)

del x_test_stwd

print("x_test pre-processing complete")

x_test cleaning start at 26:12:20



x_test cleaned
x_test tokenizing start at 26:19:18


KeyboardInterrupt: ignored

In [0]:
from googleapiclient.http import MediaFileUpload

# files = ['x_test_stwd_lmtz.csv']

def upload_to_GCS(file_name):
  file_path = '/tmp/' + file_name
  
  media = MediaFileUpload(file_path, 
                          mimetype='text/csv',
                          resumable=True)

  request = gcs_service.objects().insert(bucket=bucket_name, 
                                         name=file_name,
                                         media_body=media)

  response = None
  while response is None:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, response = request.next_chunk()
  print(file_name + ' uploaded')
    
# [upload_to_GCS(file_name) for file_name in files]

# print('Upload complete')